<a href="https://colab.research.google.com/github/zzanggyu/AlarmProject/blob/master/model_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install opencv-python-headless pillow easyocr flask flask-ngrok
!pip install flask-cors
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 26.3 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import easyocr
from flask import Flask, request, jsonify
import base64
from ultralytics import YOLO
from pyngrok import ngrok
import os
import traceback
from flask_cors import CORS
import time
from scipy import ndimage

# Flask 애플리케이션 생성 및 CORS 설정
app = Flask(__name__)
CORS(app) # 모든 경로에 대해 CORS 허용

class PillRecognitionModel:

    def __init__(self):
        self.yolo_model = self.load_yolo_model() # YOLO 모델 로드
        self.ocr_reader = easyocr.Reader(['en']) # 영어 OCR 리더 초기화
        # self.vgg_model = self.load_vgg_model()
        ## rgb 그룹 한글 색상으로 매핑 api에 한글색상이름으로 돼있음 색상이 부정확할 시 여기 수정
        self.color_groups = {
            '하양': [('하양', (210, 210, 210)), ('하양', (220, 220, 220)), ('하양', (240, 240, 240))],
            '검정': [('검정', (0, 0, 0)), ('검정', (20, 20, 20))],
            '회색': [('회색', (180, 180, 180)), ('회색', (128, 128, 128)), ('회색', (80, 80, 80))],
            '노랑/주황/분홍/빨강/갈색': [
                ('노랑', (255, 255, 0)), ('노랑', (255, 255, 100)), ('노랑', (230, 200, 50)), ('노랑', (235, 215, 140)),
                ('주황', (255, 165, 0)), ('주황', (255, 140, 0)), ('주황', (230, 135, 25)),
                ('분홍', (240, 128, 46)), ('분홍', (255, 192, 203)), ('분홍', (255, 182, 193)), ('분홍', (210, 180, 180)),
                ('빨강', (255, 0, 0)), ('빨강', (220, 20, 60)),
                ('갈색', (139, 69, 19))
            ],
            '연두/초록/청록': [
                ('연두', (154, 205, 50)), ('연두', (124, 252, 0)), ('연두', (210, 250, 210)), ('연두', (192, 217, 197)),
                ('초록', (128, 255, 0)), ('초록', (34, 139, 34)), ('초록', (60, 150, 60)),
                ('청록', (0, 255, 255)), ('청록', (0, 206, 209))
            ],
            '파랑/남색': [
                ('파랑', (135, 206, 235)), ('파랑', (100, 149, 237)), ('파랑', (0, 0, 255)), ('파랑', (30, 144, 255)),
                ('남색', (0, 0, 128)), ('남색', (25, 25, 112))
            ],
            '자주/보라': [
                ('자주', (255, 0, 255)), ('자주', (218, 112, 214)),
                ('보라', (128, 0, 128)), ('보라', (148, 0, 211))
            ]
        }

    def load_yolo_model(self):
        # YOLO 모델 로드 (경로는 실제 모델 파일 위치로 변경 필요)
        # 환경 변수에서 YOLO 모델 경로를 가져옴
        model_path = os.environ.get('YOLO_MODEL_PATH', 'G:/내 드라이브/best.pt')
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"YOLO model file not found at {model_path}")
        model = YOLO(model_path)
        return model

    def preprocess_image_for_detection(self, image):
        # YOLO 검출을 위한 이미지 전처리
        target_size = (640, 640)  # YOLO 모델의 입력 크기에 맞게 조정
        image = cv2.resize(image, target_size)
        image = image.astype(np.float32) / 255.0
        return image

    def detect_pill(self, image):
        # 전처리된 이미지 가져오기
        preprocessed_image = self.preprocess_image_for_detection(image)
        # YOLO 모델을 사용하여 이미지에서 알약 감지
        results = self.yolo_model(preprocessed_image)
        return results[0].boxes.xyxy.cpu().numpy() # 바운딩 박스 좌표 반환

    def preprocess_image_for_ocr(self, image):
        # OCR을 위한 이미지 전처리
        # 그레이스케일 변환
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # 노이즈 제거
        denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

        enhanced = clahe.apply(denoised)

        _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return binary

    # 회전 감지 및 보정
    def detect_and_correct_rotation(self, image):
        edges = cv2.Canny(image, 50, 150, apertureSize=3)
        lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

        if lines is not None:
            angles = []
            for rho, theta in lines[:, 0]:
                angle = np.degrees(theta)
                if angle < 45 or angle > 135:
                    angles.append(angle)

            if angles:
                median_angle = np.median(angles)
                rotation_angle = median_angle if median_angle < 45 else median_angle - 90
                rotated = ndimage.rotate(image, rotation_angle)
                return rotated, rotation_angle

        return image, 0

    # 최적의 앵글각도 찾기 신뢰도가 가장 높은 것이 최적의 앵글
    def apply_ocr_with_rotation(self, image):
        best_result = None
        best_confidence = 0
        best_angle = 0
        angles = [0, 90, 180, 270]

        for angle in angles:
            rotated = ndimage.rotate(image, angle)
            result = self.ocr_reader.readtext(rotated)

            if result:
                confidence = np.mean([detection[2] for detection in result if len(detection) == 3])
                if confidence > best_confidence:
                    best_confidence = confidence
                    best_result = result
                    best_angle = angle

        return best_result, best_angle

    def extract_text(self, image, bbox):
        # 감지된 알약 영역에서 텍스트 추출
        x1, y1, x2, y2 = map(int, bbox)
        pill_image = image[y1:y2, x1:x2]
        preprocessed_image = self.preprocess_image_for_ocr(pill_image)
        result = self.ocr_reader.readtext(preprocessed_image)
        return [text for _, text, _ in result] # 추출된 텍스트 목록 반환

    def process_image(self, image):

        results = []

        # 알약 감지
        bboxes = self.detect_pill(image)

        for bbox in bboxes:
            x1, y1, x2, y2 = map(int, bbox)
            pill_image = image[y1:y2, x1:x2]

            # 색상 추출
            color_name = self.extract_pill_color(image, bbox)

            # 이미지 전처리
            preprocessed = self.preprocess_image_for_ocr(pill_image)

            # 회전 감지 및 보정
            corrected, rotation_angle = self.detect_and_correct_rotation(preprocessed)

            # OCR 적용
            ocr_result, best_angle = self.apply_ocr_with_rotation(corrected)



            results.append({
#                 'bbox': bbox.tolist(),
                'text': [text for _, text, _ in ocr_result] if ocr_result else [],
                'color': color_name,
#                 'rotation_angle': rotation_angle,
#                 'ocr_rotation_angle': best_angle
            })

        return results

    ######## 색상 추출 함수들 ##########

    ## 두 색상 간의 유클리드 거리를 계산하는 함수
    def get_color_distance(self, color1, color2):
        return sum((a - b) ** 2 for a, b in zip(color1, color2)) ** 0.5

    ## 주어진 rgb색상에 가장 가까운 색상 이름과 그룹 반환
    def get_color_name(self, rgb_color):
        min_distance = float('inf')
        closest_group = '알 수 없음'
        specific_color = '알 수 없음'

        for group_name, colors in self.color_groups.items():
            for color_name, color in colors:
                distance = self.get_color_distance(rgb_color, color)
                if distance < min_distance:
                    min_distance = distance
                    closest_group = group_name
                    specific_color = color_name

        return closest_group, specific_color

    ## 알약의 색상을 추출하는 함수
    def extract_pill_color(self, image, bbox):
        x1, y1, x2, y2 = map(int, bbox)
        pill_image = image[y1:y2, x1:x2]

        ## rgb 색상 공간으로 변환
        rgb_image = cv2.cvtColor(pill_image, cv2.COLOR_BGR2RGB)
        ## LAB 색상 공간으로 변환(색상 클러스터링을 위해서)
        lab_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2LAB)
        pixels = lab_image.reshape(-1, 3)

        ## k-means 클러스터링을 사용하여 주요 색상 추출
        kmeans = KMeans(n_clusters=3, n_init=10)
        kmeans.fit(pixels)

        ## 클러스터 중심을 rgb색상으로 변환
        colors = kmeans.cluster_centers_
        colors = colors.astype(int)
        colors = [cv2.cvtColor(color.reshape(1, 1, 3).astype(np.uint8), cv2.COLOR_LAB2RGB)[0][0] for color in colors]

        ## 가장 지배적인 색상 선택
        counts = np.bincount(kmeans.labels_)
        dominant_color = colors[np.argmax(counts)]

        ## 색상 이름을 얻음
        color_name = self.get_color_name(dominant_color)
        return color_name



    ## VGG 모델 로드
    # def load_vgg_model(self):
    #     return torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

    ## VGG 모델에서 특징(모양) 추출
    # def extract_features(self, image, bbox):
    #     pill_image = image.crop(bbox)
    #     features = self.vgg_model(pill_image.unsqueeze(0))
    #     return features



    # def analyze_shape(self, features):
        ## 모양 분석 로직 (예시)
        # return "round"

    ## 알약 제형 분석 함수
    # def identify_formulation()


model = PillRecognitionModel()

@app.route('/process_image', methods=['POST'])
def process_image():
    start_time = time.time() # 처리 시작 시간 기록
    if 'image' not in request.json:
        return jsonify({'error': 'No image data'}), 400 # 이미지 데이터가 없으면 400 에러 반환

    try:
        # 1. 이미지 디코딩
        image_data = base64.b64decode(request.json['image'])
        nparr = np.frombuffer(image_data, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError("Failed to decode image")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 2. 알약 감지 및 처리
        results = model.process_image(image)

        # 3. 결과 가공
        processed_results = []
        for i, result in enumerate(results):
            processed_results.append({
                'pill_number': i+1,
                'bbox': result['bbox'],
                'text': result['text'],
                'rotation_angle': result['rotation_angle'],
                'ocr_rotation_angle': result['ocr_rotation_angle']
            })

        end_time = time.time()
        processing_time = end_time - start_time
        print(f"Total processing time: {processing_time:.2f} seconds")

        return jsonify({'results': processed_results, 'processing_time': processing_time}), 200

    except Exception as e:
        error_trace = traceback.format_exc()
        print(f"Error occurred: {str(e)}\n{error_trace}")
        return jsonify({'error': str(e), 'trace': error_trace}), 500

if __name__ == '__main__':
    print('서버가 http://localhost:5000 에서 실행 중입니다.')
    app.run(host='0.0.0.0', port=5000, debug=True) # 모든 인터페이스에서 접근 가능, 디버그 모드 활성화